In [ ]:
pip install awpy
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import pandas as pd
from awpy import Demo
import time

In [4]:
def round_score_df(round):
    df = round.pivot(index='round_num', columns='winner',values='winner').reset_index()
    df['ct'] = df['ct'].notna().astype(int)
    df['t'] = df['t'].notna().astype(int)
    # Cumulative score
    df['ct_score'] = df['ct'].cumsum()
    df['t_score'] = df['t'].cumsum()

    #current score
    df['ct_current'] = df['ct_score'].shift(fill_value=0)
    df['t_current'] = df['t_score'].shift(fill_value=0)

    # Format as "ct-t"
    df['score'] = df['ct_current'].astype(str) + "-" + df['t_current'].astype(str)
    round_score = df
    
    return round_score

In [5]:
def kills_df(kills):
    kills_df = kills[['round_num','attacker_name','weapon']]
    my_kills = kills_df[kills_df['attacker_name']=="FX"].groupby(['round_num']).count().reset_index()
    my_kills = my_kills[['round_num','attacker_name']]
    
    return my_kills

In [6]:
def my_highlights(rounds,kills):
    my_highlights = pd.merge(kills,rounds,on='round_num',how='left')[['round_num','attacker_name','score']]
    my_highlights.rename(columns={'attacker_name':'FX_kills'})

    return my_highlights

In [ ]:
file_path = r'C:\Program Files (x86)\Steam\steamapps\common\Counter-Strike Global Offensive\game\csgo\replays'
workbook_path = r'D:\\my_highlights.xlsx'

if os.path.isfile(workbook_path):
    my_highlights_df = pd.read_excel(workbook_path)
else:
    my_highlights_df = pd.DataFrame(columns=['round_num','FX_kills','score','time','map'])

list_dem = os.listdir(file_path)

#removing the files that has .info at end
demos = [s for s in list_dem if '.info' not in s]



for demo in demos:

    file_path_demo = os.path.join(file_path,demo)
    dem = Demo(file_path_demo)
    dem.parse()

    cs_map = dem.header['map_name']

    rnds = dem.rounds.to_pandas()
    klls = dem.kills.to_pandas()

    rnds_df = round_score_df(rnds)
    klls_df = kills_df(klls)

    highlight = my_highlights(rnds_df,klls_df)

    file_stats = os.stat(file_path_demo)

    highlight['time'] = time.ctime(file_stats.st_ctime)
    highlight['map'] = cs_map

    my_highlights_df = pd.concat([my_highlights_df,highlight])
    print(f'Done Parsing....{file_path_demo}')

my_highlights_df.to_excel(workbook_path, index=False)


Parsing....D:\CS2 Demos\003774033248846348588_1021889735.dem
Parsing....D:\CS2 Demos\003775157909655126359_1386198079.dem
Parsing....D:\CS2 Demos\match730_003774219150063305230_0999312015_141.dem
Parsing....D:\CS2 Demos\match730_003774406715580088795_0868543628_403.dem
Parsing....D:\CS2 Demos\match730_003774590297514705423_0798539187_142.dem
Parsing....D:\CS2 Demos\match730_003774592679074070647_0017426932_141.dem
Parsing....D:\CS2 Demos\match730_003774596845192347831_1302506794_403.dem
Parsing....D:\CS2 Demos\match730_003775145621753692463_1893614199_142.dem
Parsing....D:\CS2 Demos\match730_003775328999677362520_1319199445_141.dem
Parsing....D:\CS2 Demos\match730_003775332884475281552_1747652881_142.dem
Parsing....D:\CS2 Demos\match730_003775515620301341218_0508359405_154.dem
Parsing....D:\CS2 Demos\match730_003775521055582453889_1985575054_154.dem
